# 1. Loading Credentials from Kedro

# Configuration

> *Note:* This documentation is based on `Kedro 0.15.8`, if you spot anything that is incorrect then please create an [issue](https://github.com/quantumblacklabs/kedro/issues) or pull request.
>
> This section contains detailed information about configuration.

Relevant API documentation: [ConfigLoader](/kedro.config.ConfigLoader)

## Local and base configuration

We recommend that you keep all configuration files in the `conf` directory of a Kedro project. However, if you prefer, you may point Kedro to any other directory and change the configuration paths by overriding `CONF_ROOT` variable from the derived `ProjectContext` class in `src/<project-package>/run.py` as follows:
```python
class ProjectContext(KedroContext):
    CONF_ROOT = "new_conf"
    # ...
```

## Loading

Kedro-specific configuration (e.g., `DataCatalog` configuration for IO) is loaded using the `ConfigLoader` class:

```python
from kedro.config import ConfigLoader

conf_paths = ["conf/base", "conf/local"]
conf_loader = ConfigLoader(conf_paths)
conf_catalog = conf_loader.get("catalog*", "catalog*/**")
```

This will recursively scan for configuration files firstly in `conf/base/` and then in `conf/local/` directory according to the following rules:

* ANY of the following is true:
  * filename starts with `catalog` OR
  * file is located in a sub-directory whose name is prefixed with `catalog`
* AND file extension is one of the following: `yaml`, `yml`, `json`, `ini`, `pickle`, `xml` or `properties`

Configuration information from files stored in `base` or `local` that match these rules is merged at runtime and returned in the form of a config dictionary:

* If any 2 configuration files located inside the same environment path (`conf/base/` or `conf/local/` in this example) contain the same top-level key, `load_config` will raise a `ValueError` indicating that the duplicates are not allowed.

> *Note:* Any top-level keys that start with `_` character are considered hidden (or reserved) and therefore are ignored right after the config load. Those keys will neither trigger a key duplication error mentioned above, nor will they appear in the resulting configuration dictionary. However, you may still use such keys for various purposes. For example, as [YAML anchors and aliases](https://confluence.atlassian.com/bitbucket/yaml-anchors-960154027.html).

* If 2 configuration files have duplicate top-level keys, but are placed into different environment paths (one in `conf/base/`, another in `conf/local/`, for example) then the last loaded path (`conf/local/` in this case) takes precedence and overrides that key value. `ConfigLoader.get(<pattern>, ...)` will not raise any errors, however a `DEBUG` level log message will be emitted with the information on the over-ridden keys.
* If the same environment path is passed multiple times, a `UserWarning` will be emitted to draw attention to the duplicate loading attempt, and any subsequent loading after the first one will be skipped.

In [1]:
# Load Credentials
from kedro.config import ConfigLoader

conf_paths = ["conf/base", "conf/local"]
conf_loader = ConfigLoader(conf_paths)
conf_catalog = conf_loader.get("credentials*")

# 2. The Azure Machine Learning SDK for Python
While graphical interfaces like Azure Machine Learning studio make it easy to create and manage machine learning assets, it is often dvantageous to use a code-based approach to managing resources. By writing scripts to create and manage resources, you can:

* Automate asset creation and configuration to make it repeatable.
* Ensure consistency for resources that must be replicated in multiple environments (for example, development, test, and production)
* Incorporate machine learning asset configuration into developer operations (DevOps) workflows, such as continuous integration / continuous deployment (CI/CD) pipelines.
Azure Machine Learning provides software development kits (SDKs) for Python and R, which you can use to create, manage, and use assets in an Azure Machine Learning workspace.

> *Note:*: This course focuses on the Python SDK because it has broader capabilities than the R SDK, which is in preview at the time of writing.

# 3.Installing the Azure Machine Learning SDK for Python
You can install the Azure Machine Learning SDK for Python by using the pip package management utility, as shown in the following code sample:

```python
pip install azureml-sdk
```

The SDK includes optional extras that aren't required for core operations, but can be useful in some scenarios. For example, the notebooks extra includes widgets for displaying detailed output in Jupyter Notebooks, the automl extra includes packages for automated machine learning training, and the explain extra includes packages for generating model explanations. To install extras, specify them in brackets as shown here:

```python
pip install azureml-sdk[notebooks,automl,explain]
```

More Information: For more information about installing the Azure Machine Learning SDK for Python, see the SDK documentation. Also, you should be aware that the SDK is updated on a regular basis, and review the release notes for the latest release.

In [2]:
!pip install azureml-sdk[notebooks,automl,explain]

  Using cached six-1.14.0-py2.py3-none-any.whl (10 kB)
ERROR: kedro 0.15.8 has requirement pandas<1.0,>=0.24.0, but you'll have pandas 0.23.4 which is incompatible.


# 2. Creating a Workspace
You can create a workspace in any of the following ways:

* In the Microsoft Azure portal, create a new Machine Learning resource, specifying the subscription, resource group, workspace name, and workspace edition.
* Use the Azure Machine Learning Python SDK to run code that creates a workspace. For example:

```python
from azureml.core import Workspace
    
ws = Workspace.create(name='aml-workspace', 
                  subscription_id='123456-abc-123...',
                  resource_group='aml-resources',
                  create_resource_group=True,
                  location='eastus',
                  sku='enterprise'
                 )
```

In [3]:
from azureml.core import Workspace
from azureml.exceptions import WorkspaceException

# Create Azure Workspacefor the given subscription ID
try:
    ws = Workspace.create(name='azml-workspace', 
                          subscription_id=conf_catalog['azure_ml']['subscription_id'],
                          resource_group='aml-resources',
                          create_resource_group=True,
                          location='eastus',
                          sku='enterprise'
                         )
    ws.write_config(file_name="ws_config.json")
except WorkspaceException :
    print('Workspace Already Exists, hence not created.')
    print('Loading Worksapce from config')

    # Load Worskapce from config
    ws = Workspace.from_config()
    
    # Alternative , load from azure ml
    '''
    ws = Workspace.get(name="azml-workspace",
                       subscription_id=conf_catalog['azure_ml']['subscription_id'],
                       resource_group='aml-resources')
    '''

   

/home/yuvraj/anaconda3/envs/AzureDS/lib/python3.6/site-packages/azureml/data/dataset_type_definitions.py:69: DeprecationWarning: FileType Enum is Deprecated in > 1.0.39. Use strings instead.
  category=DeprecationWarning)


2020-04-03 00:41:14,188 - azureml.core - WARNING - Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception [Errno 2] No such file or directory: '/home/yuvraj/.local/lib/python3.6/site-packages/six-1.12.0.dist-info/METADATA'.
Workspace Already Exists, hence not created.
Loading Worksapce from config
2020-04-03 00:41:15,329 - azureml.core.workspace - INFO - Found the config file in: /home/yuvraj/projects/AzureAworkspace/azureds/.azureml/config.json
